In [1]:
import numpy as np
import pandas as pd
import itertools
from pandas import DataFrame
import re
import os
data= pd.read_csv('../input/train.csv')

In [2]:
data.loc[data['target']<0.48, 'target']=0
data.loc[data['target']>=0.48, 'target']=1
data['target']= data['target'].round(decimals=0)
x= data['comment_text']
targets= data['target']

In [3]:
from sklearn.model_selection import train_test_split as tts


In [4]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding, Dropout, MaxPooling1D, Conv1D, AveragePooling1D, Flatten,Bidirectional,BatchNormalization, LSTM, SpatialDropout1D
from keras.initializers import TruncatedNormal

Using TensorFlow backend.


In [5]:
def reduce_lengthening(text):

    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [6]:
tokenizer = Tokenizer( filters='!"#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n', lower=True)
test_data= pd.read_csv('../input/test.csv')

x=[reduce_lengthening(i) for i in x]
tokenizer.fit_on_texts(x)

vocab_size = len(tokenizer.word_index) + 1


In [7]:

x_train, x_test, y_train, y_test= tts(x,targets,test_size=0.2, random_state=4)
x_tr=tokenizer.texts_to_sequences(x_train)
x_ts=tokenizer.texts_to_sequences(x_test)
x_tr=pad_sequences(x_tr, maxlen=250)
x_ts=pad_sequences(x_ts, maxlen=250)


In [8]:
embedding_dim = 20

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim= embedding_dim, input_length=250, embeddings_initializer='TruncatedNormal'))
#model.add(SpatialDropout1D(0.2))
model.add(AveragePooling1D(pool_size=2))
model.add(MaxPooling1D(pool_size=2))

model.add((GRU(256, recurrent_dropout=0.2)))
#model.add(Conv1D(100, kernel_size=8, activation='relu'))
#model.add(GlobalMaxPooling1D())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 20)           8782120   
_________________________________________________________________
average_pooling1d_1 (Average (None, 125, 20)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 62, 20)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               212736    
_________________________________________________________________
dense_1 (Dense)              (None, 300)               77100     
_________________________________________________________________
dropout_1 (Dropout)  

In [9]:
model.fit(x_tr, y_train,epochs=3,verbose=1,batch_size=300)
_, val_acc1 = model.evaluate(x_ts, y_test, verbose=0)
print('val_acc is: ', val_acc1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
1443899/1443899 [==============================] - 400s 277us/step - loss: 0.1834 - acc: 0.9379
Epoch 2/3
 195300/1443899 [===>..........................] - ETA: 5:44 - loss: 0.1502 - acc: 0.9454

In [10]:
embedding_dim = 50

model2 = Sequential()
model2.add(Embedding(input_dim=vocab_size, output_dim= embedding_dim, input_length=250, embeddings_initializer='TruncatedNormal'))
model2.add(Conv1D(200, kernel_size=10,strides=1,activation='relu' ))
model2.add(AveragePooling1D(pool_size=2, strides=None, padding='same'))
model2.add(MaxPooling1D(pool_size=5, strides=None, padding='same'))
model2.add(GRU(256, recurrent_dropout=0.2))
model2.add(Dropout(0.3))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 50)           21955300  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 241, 200)          100200    
_________________________________________________________________
average_pooling1d_2 (Average (None, 121, 200)          0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 25, 200)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               350976    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total para

In [11]:
x_train, x_test, y_train, y_test= tts(x,targets,test_size=0.2, random_state=12)
x_tr=tokenizer.texts_to_sequences(x_train)
x_ts=tokenizer.texts_to_sequences(x_test)
x_tr=pad_sequences(x_tr, maxlen=250)
x_ts=pad_sequences(x_ts, maxlen=250)


In [12]:
model2.fit(x_tr, y_train,epochs=2,verbose=1,batch_size=300)
_, val_acc2 = model2.evaluate(x_ts, y_test, verbose=0)
print('val_acc is: ', val_acc2)

Epoch 1/2
1443899/1443899 [==============================] - 260s 180us/step - loss: 0.1410 - acc: 0.9474
Epoch 2/2
1443899/1443899 [==============================] - 256s 177us/step - loss: 0.1098 - acc: 0.9561
val_acc is:  0.9475337627262808


In [13]:
embedding_dim = 30

model3 = Sequential()
model3.add(Embedding(input_dim=vocab_size, output_dim= embedding_dim, input_length=250, embeddings_initializer='TruncatedNormal'))
model3.add(Conv1D(200, kernel_size=10,strides=1,activation='relu' ))
model3.add(AveragePooling1D(pool_size=2, strides=None, padding='same'))
model3.add(MaxPooling1D(pool_size=5, strides=None, padding='same'))
model3.add(GRU(256, recurrent_dropout=0.2))
model3.add(Dropout(0.3))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 30)           13173180  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 241, 200)          60200     
_________________________________________________________________
average_pooling1d_3 (Average (None, 121, 200)          0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 25, 200)           0         
_________________________________________________________________
gru_3 (GRU)                  (None, 256)               350976    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total para

In [14]:
x_train, x_test, y_train, y_test= tts(x,targets,test_size=0.25, random_state=24)
x_tr=tokenizer.texts_to_sequences(x_train)
x_ts=tokenizer.texts_to_sequences(x_test)
x_tr=pad_sequences(x_tr, maxlen=250, padding='post', truncating='post')
x_ts=pad_sequences(x_ts, maxlen=250, padding='post', truncating='post')


In [15]:
model3.fit(x_tr, y_train,epochs=2,verbose=1,batch_size=300)
_, val_acc3 = model3.evaluate(x_ts, y_test, verbose=0)
print('val_acc is: ', val_acc3)


Epoch 1/2
1353655/1353655 [==============================] - 210s 155us/step - loss: 0.1484 - acc: 0.9457
Epoch 2/2
1353655/1353655 [==============================] - 209s 154us/step - loss: 0.1133 - acc: 0.9548
val_acc is:  0.9505317816856498


In [16]:


embedding_dim = 20

model4 = Sequential()
model4.add(Embedding(input_dim=vocab_size, output_dim= embedding_dim, input_length=250, embeddings_initializer='TruncatedNormal'))
model4.add(Conv1D(200, kernel_size=10,strides=1 ))
model4.add(AveragePooling1D(pool_size=2, strides=None, padding='same'))
model4.add(MaxPooling1D(pool_size=5, strides=None, padding='same'))
model4.add(LSTM(256, recurrent_dropout=0.2))
model4.add(Dropout(0.3))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 250, 20)           8782120   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 241, 200)          40200     
_________________________________________________________________
average_pooling1d_4 (Average (None, 121, 200)          0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 25, 200)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               467968    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total para

In [17]:
x_train, x_test, y_train, y_test= tts(x,targets,test_size=0.15, random_state=122)
x_tr=tokenizer.texts_to_sequences(x_train)
x_ts=tokenizer.texts_to_sequences(x_test)
x_tr=pad_sequences(x_tr, maxlen=250, padding='post', truncating='post')
x_ts=pad_sequences(x_ts, maxlen=250, padding='post', truncating='post')


In [18]:
model4.fit(x_tr, y_train,epochs=2,verbose=1,batch_size=300)
_, val_acc4 = model4.evaluate(x_ts, y_test, verbose=0)
print('val_acc is: ', val_acc4)


Epoch 1/2
1534142/1534142 [==============================] - 254s 166us/step - loss: 0.2436 - acc: 0.9263
Epoch 2/2
1534142/1534142 [==============================] - 252s 164us/step - loss: 0.1276 - acc: 0.9506
val_acc is:  0.949470324896946


In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics

In [20]:
vectorizer = CountVectorizer(stop_words='english', decode_error='ignore')
#x_train= vectorizer.fit_transform(x_train.values)
clf =  LogisticRegression()
clf.fit(x_tr, y_train)
#x_test=vectorizer.transform(x_test)
pred = clf.predict(x_ts)
val_acc5 = metrics.accuracy_score(y_test, pred)



/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [21]:


coef1=0.892
coef2=0.895
coef3=0.897
coef4=0.895
coef5=0.75



In [22]:

test=test_data['comment_text']
test=[reduce_lengthening(i) for i in test]
#testx=vectorizer.transform(test)
test=tokenizer.texts_to_sequences(test)
test=pad_sequences(test, maxlen=250, padding='post', truncating='post')
pred1=model.predict(test, batch_size=300)
pred2=model2.predict(test, batch_size=300)
pred3=model3.predict(test, batch_size=300)
pred4=model4.predict(test, batch_size=300)

pred5=clf.predict(test)
pred1=[pred1[i]*val_acc1*coef1 for i in range(len(pred1))]
pred2=[pred2[i]*val_acc2*coef2 for i in range(len(pred2))]
pred3=[pred3[i]*val_acc3*coef3 for i in range(len(pred3))]
pred4=[pred4[i]*val_acc4*coef4 for i in range(len(pred4))]
pred5=[pred5[i]*val_acc5*coef5 for i in range(len(pred5))]
prediction=[(pred1[i]+pred2[i]+pred3[i]+pred4[i]+pred5[i])/(coef1*val_acc1+coef2*val_acc2+coef3*val_acc3+coef4*val_acc4+coef5*val_acc5) for i in range(len(pred1))]
prediction = list(itertools.chain(*prediction))
test_data['prediction']=prediction



In [23]:


test_data=test_data.drop('comment_text', axis=1)
test_data.head(30)



,id,prediction
0,7000000,0.023338
1,7000001,0.018125
2,7000002,0.018746
3,7000003,0.021774
4,7000004,0.414110
5,7000005,0.017248
6,7000006,0.017479
7,7000007,0.020903
8,7000008,0.022590
9,7000009,0.018754


In [24]:
test_data.head(30)

,id,prediction
0,7000000,0.023338
1,7000001,0.018125
2,7000002,0.018746
3,7000003,0.021774
4,7000004,0.414110
5,7000005,0.017248
6,7000006,0.017479
7,7000007,0.020903
8,7000008,0.022590
9,7000009,0.018754


In [25]:

test_data.to_csv('submission.csv', index=False)